# SESIÓN 2: Inyección de Dependencias y Arquitectura Modular

**Objetivo:** Aprender a organizar aplicaciones FastAPI mediante inyección de dependencias, routers modulares y patrones de arquitectura escalables.

**Duración:** 180 minutos

**Contenido:**
1. Sistema de Dependencias (Inversión de control, Depends(), documentación)
2. Dependencias con yield (Gestión de recursos, setup/teardown)
3. Clases como Servicios (Service Layer pattern)
4. APIRouter y Modularización (Organización por dominio)
5. Sub-dependencias (Composición y grafo de resolución)

## CONFIGURACIÓN DEL ENTORNO

Ejecuta estas celdas para preparar el entorno de trabajo.

In [ ]:
# Verificación de Python
import sys
print(f"Python: {sys.version}")
assert sys.version_info >= (3, 8), "Se requiere Python 3.8 o superior"

In [ ]:
# Instalación de dependencias
!pip install fastapi==0.115.0 uvicorn[standard]==0.32.0 -q
print("Dependencias instaladas")

In [ ]:
# Importaciones globales
from fastapi import FastAPI, Depends, HTTPException, Header, APIRouter, status
from fastapi.testclient import TestClient
from pydantic import BaseModel, Field
from typing import Optional, Annotated, Generator
import time

print("Imports completados")

---

## 1. SISTEMA DE DEPENDENCIAS

### ¿Qué es la Inyección de Dependencias?

La inyección de dependencias (DI) es un patrón de diseño donde las funciones reciben sus dependencias desde el exterior, en lugar de crearlas internamente.

**Analogía:** Es como pedir café en una cafetería:
- **Sin DI:** Entras a la cocina, mueles el café, preparas el agua, haces el café tú mismo
- **Con DI:** Le pides al barista (FastAPI) que te prepare el café. Él se encarga de todo

**Ventajas:**
- Código más limpio y testeable
- Reutilización de lógica
- Menos repetición (DRY: Don't Repeat Yourself)
- FastAPI gestiona automáticamente el ciclo de vida

### Annotated: El Patrón Moderno (Python 3.9+)

**Evolución del patrón:**

FastAPI ha evolucionado en cómo se declaran dependencias:

```python
# ❌ ESTILO ANTIGUO (pre-2023, aún funciona pero verboso)
def endpoint(token: str = Depends(verificar_token)):
    ...

# ✅ ESTILO MODERNO (recomendado desde FastAPI 0.95+)
TokenDep = Annotated[str, Depends(verificar_token)]

def endpoint(token: TokenDep):
    ...
```

**¿Por qué `Annotated`?**

1. **Reutilización:** Defines el tipo una vez, lo usas en múltiples lugares
2. **Legibilidad:** `token: TokenDep` es más claro que `token: str = Depends(...)`
3. **Soporte IDE:** Mejor autocompletado y detección de errores
4. **Separación de responsabilidades:** El tipo está separado de su implementación

**Sintaxis:**
```python
Annotated[TipoBase, Metadato1, Metadato2, ...]
#         ^^^^^^^^  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
#         Tipo real  Información adicional (dependencias, validaciones, etc.)
```

**Nota histórica:** Si vienes de la Sesión 1 o de tutoriales antiguos y viste `= Depends(...)`, no te preocupes. Ambos estilos funcionan, pero en este curso usamos el patrón moderno para prepararte para proyectos profesionales actuales.

### Ejemplo SIN Inyección de Dependencias

Imagina que queremos validar un token en múltiples endpoints. Sin DI, repetiríamos código:

In [ ]:
import uvicorn
import threading
import time
app = FastAPI()

@app.get("/usuarios")
def obtener_usuarios(token: str = Header(...)):
    # Validación repetida
    if token != "secreto123":
        raise HTTPException(status_code=401, detail="Token inválido")
    return {"usuarios": ["Ana", "Luis"]}

@app.get("/productos")
def obtener_productos(token: str = Header(...)):
    # Misma validación repetida ❌
    if token != "secreto123":
        raise HTTPException(status_code=401, detail="Token inválido")
    return {"productos": ["Laptop", "Mouse"]}

client = TestClient(app)
response = client.get("/usuarios", headers={"token": "secreto123"})
print(response.json())

# Servidor en puerto 8300
def run_server_8300():
    uvicorn.run(app, host="127.0.0.1", port=8300, log_level="warning")

server_thread = threading.Thread(target=run_server_8300, daemon=True)
server_thread.start()
time.sleep(2)

print(f"Servidor iniciado en http://127.0.0.1:8300")
print(f"Documentacion: http://127.0.0.1:8300/docs")


**Problema:** Estamos duplicando la lógica de validación. Si cambiamos el token, debemos modificar múltiples lugares.

### Ejemplo CON Inyección de Dependencias

Usamos `Depends()` para extraer la lógica de validación a una función reutilizable:

In [ ]:
# DEPENDENCIA: Función que valida el token
import uvicorn
import threading
import time
def verificar_token(token: str = Header(...)):
    if token != "secreto123":
        raise HTTPException(status_code=401, detail="Token inválido")
    return token  # Devolvemos el token validado

# Definimos un tipo reutilizable con Annotated
TokenValidado = Annotated[str, Depends(verificar_token)]

app = FastAPI()

@app.get("/usuarios")
def obtener_usuarios(token_validado: TokenValidado):
    # FastAPI automáticamente llama a verificar_token()
    # Mucho más limpio que: token_validado: str = Depends(verificar_token)
    return {"usuarios": ["Ana", "Luis"], "autenticado_con": token_validado}

@app.get("/productos")
def obtener_productos(token_validado: TokenValidado):
    # Reutilizamos el tipo TokenValidado ✅
    return {"productos": ["Laptop", "Mouse"]}

# Prueba
client = TestClient(app)
response = client.get("/usuarios", headers={"token": "secreto123"})
print(response.json())

# Servidor en puerto 8301
def run_server_8301():
    uvicorn.run(app, host="127.0.0.1", port=8301, log_level="warning")

server_thread = threading.Thread(target=run_server_8301, daemon=True)
server_thread.start()
time.sleep(2)

print(f"Servidor iniciado en http://127.0.0.1:8301")
print(f"Documentacion: http://127.0.0.1:8301/docs")


### Inversión de Control

**Principio clave:** FastAPI "invierte el control" del flujo del programa.

- **Sin DI:** Tu endpoint llama a las funciones que necesita
- **Con DI:** FastAPI llama a las dependencias *antes* de ejecutar tu endpoint

**Flujo de ejecución:**
```
1. Usuario hace request → /usuarios
2. FastAPI detecta Depends(verificar_token)
3. FastAPI ejecuta verificar_token()
4. Si no hay excepción, FastAPI ejecuta obtener_usuarios()
5. Devuelve respuesta
```

### Documentación Automática

FastAPI incluye automáticamente las dependencias en la documentación interactiva. Ejecuta el siguiente código y visita `http://127.0.0.1:8000/docs` (si estuvieras ejecutando con uvicorn):

In [ ]:
# En Jupyter usamos TestClient para simular requests
# En producción, las dependencias aparecen en /docs automáticamente

import uvicorn
import threading
import time
def obtener_usuario_actual(token: str = Header(..., description="Token de autenticación")):
    """Valida el token y devuelve el usuario."""
    if token != "secreto123":
        raise HTTPException(status_code=401, detail="No autorizado")
    return {"username": "admin", "rol": "administrador"}

# Tipo reutilizable
UsuarioActual = Annotated[dict, Depends(obtener_usuario_actual)]

app = FastAPI(title="API con Dependencias")

@app.get("/perfil")
def obtener_perfil(usuario: UsuarioActual):
    """Obtiene el perfil del usuario autenticado."""
    return {"mensaje": f"Bienvenido {usuario['username']}", "datos": usuario}

# Simulación de request
client = TestClient(app)
response = client.get("/perfil", headers={"token": "secreto123"})
print(response.json())

# Servidor en puerto 8302
def run_server_8302():
    uvicorn.run(app, host="127.0.0.1", port=8302, log_level="warning")

server_thread = threading.Thread(target=run_server_8302, daemon=True)
server_thread.start()
time.sleep(2)

print(f"Servidor iniciado en http://127.0.0.1:8302")
print(f"Documentacion: http://127.0.0.1:8302/docs")


**Observación:** En `/docs`, FastAPI muestra:
- El parámetro `token` como header requerido
- La descripción de la dependencia
- El tipo de datos esperado

### Micro-reto: Inyección de Dependencias (Clásico vs Annotated)

**Objetivo**
Implementar la misma lógica de paginación utilizando dos sintaxis diferentes de FastAPI: la clásica con `Depends` en los parámetros de la función y la moderna utilizando `Annotated` para crear tipos reutilizables.

**Instrucciones**

1. **Define la lógica:** Crea una función `paginar` que reciba `skip` (default 0) y `limit` (default 10). Si `limit` es mayor a 100, debe lanzar una `HTTPException`. Devuelve un diccionario.
2. **Caso A (Clásico):** Crea el endpoint `/items-clasico` inyectando la dependencia directamente en el argumento de la función: `params: dict = Depends(paginar)`.
3. **Caso B (Annotated):** Crea un alias de tipo llamado `PaginacionDep` usando `Annotated` y `Depends`. Luego, crea el endpoint `/items-annotated` usando este nuevo tipo en los argumentos: `params: PaginacionDep`.
4. **Verificación:** Ambos endpoints deben comportarse exactamente igual.


In [4]:
import uvicorn
import threading
import time
from typing import Annotated
from fastapi import FastAPI, Depends, HTTPException
from fastapi.testclient import TestClient

app = FastAPI()

# 1. Lógica de la dependencia
def paginar(skip: int = 0, limit: int = 10):
    # TODO: Validar que limit <= 100. Si no, raise HTTPException 400.
    # TODO: Retornar diccionario con skip y limit.
    pass

# 2. Caso A: Uso clásico de Depends
@app.get("/items-clasico")
def listar_items_clasico(params: dict = Depends(paginar)):
    return {"estilo": "clasico", "datos": params}

# 3. Caso B: Uso de Annotated (Best Practice moderna)
# TODO: Definir el tipo PaginacionDep usando Annotated[dict, Depends(paginar)]
PaginacionDep = None # Reemplaza None por la definición correcta

@app.get("/items-annotated")
def listar_items_annotated(params: dict): # TODO: Cambiar el tipo 'dict' por 'PaginacionDep'
    return {"estilo": "annotated", "datos": params}

# --- Bloque de Pruebas ---
client = TestClient(app)

print("--- Probando Endpoint Clásico ---")
print(client.get("/items-clasico?skip=5&limit=50").json())

print("\n--- Probando Endpoint Annotated ---")
print(client.get("/items-annotated?skip=5&limit=50").json())

print("\n--- Probando Validación de Error (debe fallar en ambos) ---")
print(client.get("/items-annotated?limit=150").json())

# Servidor en puerto 8303
def run_server_8303():
    uvicorn.run(app, host="127.0.0.1", port=8303, log_level="critical")

if __name__ == "__main__": # Asegura que no se ejecute dos veces si importas
    server_thread = threading.Thread(target=run_server_8303, daemon=True)
    server_thread.start()
    time.sleep(2)

    print(f"\n✅ Servidor iniciado en http://127.0.0.1:8303")
    print(f"📄 Documentacion: http://127.0.0.1:8303/docs")

--- Probando Endpoint Clásico ---
{'estilo': 'clasico', 'datos': None}

--- Probando Endpoint Annotated ---
{'detail': [{'type': 'missing', 'loc': ['body'], 'msg': 'Field required', 'input': None}]}

--- Probando Validación de Error (debe fallar en ambos) ---
{'detail': [{'type': 'missing', 'loc': ['body'], 'msg': 'Field required', 'input': None}]}

✅ Servidor iniciado en http://127.0.0.1:8303
📄 Documentacion: http://127.0.0.1:8303/docs


---

## 2. DEPENDENCIAS CON YIELD

### Gestión de Recursos

Cuando trabajamos con recursos que necesitan **abrirse y cerrarse** (bases de datos, archivos, conexiones), usamos dependencias con `yield`.

**Analogía:** Es como pedir prestado un libro en la biblioteca:
1. **Setup:** Sacas el libro del estante (abres conexión)
2. **Uso:** Lees el libro (usas la dependencia en tu endpoint)
3. **Teardown:** Devuelves el libro (cierras conexión)

**Patrón:**
```python
def mi_dependencia():
    # Setup: código ANTES del yield
    recurso = abrir_recurso()
    try:
        yield recurso  # Se inyecta en el endpoint
    finally:
        # Teardown: código DESPUÉS del yield
        cerrar_recurso(recurso)
```

### Ejemplo: Simulación de Conexión a Base de Datos

Vamos a simular una conexión a BD que debe abrirse y cerrarse automáticamente:

In [ ]:
# Simulamos una clase de conexión a BD
import uvicorn
import threading
import time
class FakeDatabase:
    def __init__(self):
        self.connected = False
        self.data = {"usuarios": ["Ana", "Luis", "María"]}
    
    def connect(self):
        print("🔌 Conectando a la base de datos...")
        self.connected = True
    
    def disconnect(self):
        print("🔌 Cerrando conexión a la base de datos...")
        self.connected = False
    
    def query(self, table: str):
        if not self.connected:
            raise Exception("No conectado a la BD")
        return self.data.get(table, [])

# DEPENDENCIA CON YIELD
def obtener_db() -> Generator[FakeDatabase, None, None]:
    db = FakeDatabase()
    db.connect()  # Setup: abrimos conexión
    try:
        yield db  # Inyectamos la BD en el endpoint
    finally:
        db.disconnect()  # Teardown: cerramos conexión (siempre se ejecuta)

# Tipo reutilizable para la BD
DatabaseDep = Annotated[FakeDatabase, Depends(obtener_db)]

app = FastAPI()

@app.get("/usuarios")
def listar_usuarios(db: DatabaseDep):
    # FastAPI ya ejecutó connect() antes de llegar aquí
    usuarios = db.query("usuarios")
    return {"usuarios": usuarios}
    # Cuando termina, FastAPI ejecuta disconnect() automáticamente

# Prueba
client = TestClient(app)
response = client.get("/usuarios")
print(response.json())

# Servidor en puerto 8304
def run_server_8304():
    uvicorn.run(app, host="127.0.0.1", port=8304, log_level="warning")

server_thread = threading.Thread(target=run_server_8304, daemon=True)
server_thread.start()
time.sleep(2)

print(f"Servidor iniciado en http://127.0.0.1:8304")
print(f"Documentacion: http://127.0.0.1:8304/docs")


**Flujo de ejecución:**
```
1. Request → /usuarios
2. FastAPI ejecuta obtener_db()
   → db.connect() ✅
   → yield db (pausa la dependencia)
3. FastAPI ejecuta listar_usuarios(db)
4. Endpoint devuelve respuesta
5. FastAPI retoma obtener_db()
   → finally: db.disconnect() ✅
```

### Setup/Teardown Automático

El bloque `finally` **siempre se ejecuta**, incluso si el endpoint lanza una excepción:

In [ ]:
import uvicorn
import threading
import time
def obtener_db_segura() -> Generator[FakeDatabase, None, None]:
    db = FakeDatabase()
    db.connect()
    try:
        yield db
    finally:
        # Esto se ejecuta SIEMPRE, incluso si hay error
        db.disconnect()
        print("✅ Limpieza completada")

DatabaseSegura = Annotated[FakeDatabase, Depends(obtener_db_segura)]

app = FastAPI()

@app.get("/error")
def endpoint_con_error(db: DatabaseSegura):
    raise HTTPException(status_code=500, detail="Error simulado")
    # ⚠️ Nunca llegamos aquí, pero disconnect() SÍ se ejecuta

# Prueba
client = TestClient(app)
try:
    response = client.get("/error")
except Exception as e:
    print(f"Error capturado: {e}")
# Observa que "Cerrando conexión" se imprimió de todas formas

# Servidor en puerto 8305
def run_server_8305():
    uvicorn.run(app, host="127.0.0.1", port=8305, log_level="warning")

server_thread = threading.Thread(target=run_server_8305, daemon=True)
server_thread.start()
time.sleep(2)

print(f"Servidor iniciado en http://127.0.0.1:8305")
print(f"Documentacion: http://127.0.0.1:8305/docs")


### Uso con async/await

Si tu recurso es asíncrono (por ejemplo, una conexión a BD async), usa `async def` y `async with`:

In [ ]:
import uvicorn
import threading
import time
import asyncio

class AsyncFakeDatabase:
    async def connect(self):
        print("🔌 Conectando (async)...")
        await asyncio.sleep(0.1)  # Simula operación async
    
    async def disconnect(self):
        print("🔌 Desconectando (async)...")
        await asyncio.sleep(0.1)
    
    async def query(self, table: str):
        await asyncio.sleep(0.05)
        return ["dato1", "dato2"]

# DEPENDENCIA ASYNC CON YIELD
async def obtener_db_async():
    db = AsyncFakeDatabase()
    await db.connect()  # await en setup
    try:
        yield db
    finally:
        await db.disconnect()  # await en teardown

AsyncDatabaseDep = Annotated[AsyncFakeDatabase, Depends(obtener_db_async)]

app = FastAPI()

@app.get("/async-data")
async def obtener_datos(db: AsyncDatabaseDep):
    # Endpoint async puede usar await
    datos = await db.query("tabla")
    return {"datos": datos}

# Prueba
client = TestClient(app)
response = client.get("/async-data")
print(response.json())

# Servidor en puerto 8306
def run_server_8306():
    uvicorn.run(app, host="127.0.0.1", port=8306, log_level="warning")

server_thread = threading.Thread(target=run_server_8306, daemon=True)
server_thread.start()
time.sleep(2)

print(f"Servidor iniciado en http://127.0.0.1:8306")
print(f"Documentacion: http://127.0.0.1:8306/docs")


**Regla:**
- Si la dependencia usa `async def`, el endpoint **debe** ser `async def`
- Si la dependencia usa `def`, el endpoint puede ser `def` o `async def`

### 🎯 MICRO-RETO 2: Dependencia de Archivo con Yield

Crea una dependencia `abrir_log()` que:
- Simule abrir un archivo de log (puedes usar una lista en memoria)
- En setup: imprima "📄 Abriendo archivo log..."
- Haga `yield` de un objeto que tenga método `.escribir(mensaje)`
- En teardown: imprima "📄 Cerrando archivo log..."
- Úsala en un endpoint `/registrar` que escriba un mensaje en el log

In [ ]:
# TODO: Implementa la clase y la dependencia
import uvicorn
import threading
import time
class FakeLogFile:
    def __init__(self):
        self.logs = []
    
    def escribir(self, mensaje: str):
        pass  # Tu código aquí

def abrir_log():
    pass  # Tu código aquí (usa yield)

app = FastAPI()

# TODO: Crea el endpoint /registrar
@app.post("/registrar")
def registrar_evento():
    pass  # Tu código aquí

# Prueba (descomenta cuando termines)
# client = TestClient(app)
# print(client.post("/registrar", json={"evento": "Usuario login"}).json())

# Servidor en puerto 8307
def run_server_8307():
    uvicorn.run(app, host="127.0.0.1", port=8307, log_level="warning")

server_thread = threading.Thread(target=run_server_8307, daemon=True)
server_thread.start()
time.sleep(2)

print(f"Servidor iniciado en http://127.0.0.1:8307")
print(f"Documentacion: http://127.0.0.1:8307/docs")


---

## 3. CLASES COMO SERVICIOS

### Service Layer Pattern

En aplicaciones complejas, la lógica de negocio debe separarse de los endpoints. El patrón **Service Layer** encapsula esta lógica en clases reutilizables.

**Analogía:** Es como organizar un restaurante:
- **Endpoint (camarero):** Recibe pedidos, devuelve platos
- **Servicio (cocina):** Prepara los platos con lógica compleja
- **Repositorio (despensa):** Accede a los datos (ingredientes)

**Arquitectura en capas:**
```
Router (endpoints)  →  Service (lógica de negocio)  →  Repository (acceso a datos)
```

### Ejemplo: Servicio de Usuarios

Vamos a crear un servicio que gestione usuarios:

In [ ]:
# SERVICIO: Lógica de negocio
import uvicorn
import threading
import time
class UsuarioService:
    def __init__(self):
        # En producción, aquí inyectarías un Repository
        self.usuarios = {
            1: {"id": 1, "nombre": "Ana", "email": "ana@example.com"},
            2: {"id": 2, "nombre": "Luis", "email": "luis@example.com"}
        }
    
    def obtener_usuario(self, user_id: int) -> dict:
        """Lógica de negocio: buscar usuario por ID."""
        usuario = self.usuarios.get(user_id)
        if not usuario:
            raise HTTPException(status_code=404, detail="Usuario no encontrado")
        return usuario
    
    def crear_usuario(self, nombre: str, email: str) -> dict:
        """Lógica de negocio: validar y crear usuario."""
        # Validación de negocio
        if "@" not in email:
            raise HTTPException(status_code=400, detail="Email inválido")
        
        nuevo_id = max(self.usuarios.keys()) + 1
        nuevo_usuario = {"id": nuevo_id, "nombre": nombre, "email": email}
        self.usuarios[nuevo_id] = nuevo_usuario
        return nuevo_usuario

# Dependencia que inyecta el servicio
def obtener_usuario_service() -> UsuarioService:
    return UsuarioService()

# Tipo reutilizable
UsuarioServiceDep = Annotated[UsuarioService, Depends(obtener_usuario_service)]

app = FastAPI()

@app.get("/usuarios/{user_id}")
def obtener_usuario(user_id: int, service: UsuarioServiceDep):
    # El endpoint solo coordina, no tiene lógica de negocio
    return service.obtener_usuario(user_id)

@app.post("/usuarios")
def crear_usuario(nombre: str, email: str, service: UsuarioServiceDep):
    return service.crear_usuario(nombre, email)

# Pruebas
client = TestClient(app)
print(client.get("/usuarios/1").json())
print(client.post("/usuarios?nombre=Pedro&email=pedro@example.com").json())

# Servidor en puerto 8308
def run_server_8308():
    uvicorn.run(app, host="127.0.0.1", port=8308, log_level="warning")

server_thread = threading.Thread(target=run_server_8308, daemon=True)
server_thread.start()
time.sleep(2)

print(f"Servidor iniciado en http://127.0.0.1:8308")
print(f"Documentacion: http://127.0.0.1:8308/docs")


**Ventajas del Service Layer:**
- **Testeable:** Puedes testear `UsuarioService` sin FastAPI
- **Reutilizable:** Múltiples endpoints pueden usar el mismo servicio
- **Separación de responsabilidades:** Endpoints delgados, servicios gruesos

### Inyección de Clases con Parámetros

Podemos hacer que las clases sean directamente inyectables usando `__init__` con valores por defecto:

In [ ]:
# Clase inyectable directamente
import uvicorn
import threading
import time
class ConfigService:
    def __init__(self, api_key: str = Header(..., description="API Key de autenticación")):
        self.api_key = api_key
        self.config = {"max_requests": 100, "timeout": 30}
    
    def obtener_limite(self) -> int:
        # Lógica de negocio que usa la config
        return self.config["max_requests"]
    
    def validar_permiso(self) -> bool:
        return self.api_key == "clave-secreta"

# Tipo reutilizable (ConfigService se instancia automáticamente)
ConfigDep = Annotated[ConfigService, Depends()]

app = FastAPI()

@app.get("/config/limite")
def obtener_limite(service: ConfigDep):
    # FastAPI instancia ConfigService automáticamente
    # y extrae api_key del header
    if not service.validar_permiso():
        raise HTTPException(status_code=403, detail="API Key inválida")
    return {"limite": service.obtener_limite()}

# Prueba
client = TestClient(app)
response = client.get("/config/limite", headers={"api-key": "clave-secreta"})
print(response.json())

# Servidor en puerto 8309
def run_server_8309():
    uvicorn.run(app, host="127.0.0.1", port=8309, log_level="warning")

server_thread = threading.Thread(target=run_server_8309, daemon=True)
server_thread.start()
time.sleep(2)

print(f"Servidor iniciado en http://127.0.0.1:8309")
print(f"Documentacion: http://127.0.0.1:8309/docs")


**Observación:** FastAPI detecta los parámetros de `__init__` y los trata como dependencias.

### 🎯 MICRO-RETO 3: Servicio de Productos

Crea una clase `ProductoService` que:
- Tenga un diccionario de productos: `{1: {"nombre": "Laptop", "precio": 1000}, ...}`
- Método `buscar_por_precio_max(max_precio: int)`: devuelve productos con precio <= max_precio
- Método `aplicar_descuento(producto_id: int, descuento: float)`: reduce el precio en el % indicado
- Crea dos endpoints: `/productos/baratos` y `/productos/{id}/descuento`

In [ ]:
# TODO: Implementa el servicio
import uvicorn
import threading
import time
class ProductoService:
    def __init__(self):
        self.productos = {
            1: {"id": 1, "nombre": "Laptop", "precio": 1000},
            2: {"id": 2, "nombre": "Mouse", "precio": 25},
            3: {"id": 3, "nombre": "Teclado", "precio": 75}
        }
    
    def buscar_por_precio_max(self, max_precio: int):
        pass  # Tu código aquí
    
    def aplicar_descuento(self, producto_id: int, descuento: float):
        pass  # Tu código aquí

def obtener_producto_service():
    return ProductoService()

app = FastAPI()

# TODO: Implementa los endpoints
@app.get("/productos/baratos")
def productos_baratos():
    pass

@app.put("/productos/{producto_id}/descuento")
def aplicar_descuento():
    pass

# Prueba (descomenta cuando termines)
# client = TestClient(app)
# print(client.get("/productos/baratos?max_precio=100").json())
# print(client.put("/productos/1/descuento?descuento=10").json())

# Servidor en puerto 8310
def run_server_8310():
    uvicorn.run(app, host="127.0.0.1", port=8310, log_level="warning")

server_thread = threading.Thread(target=run_server_8310, daemon=True)
server_thread.start()
time.sleep(2)

print(f"Servidor iniciado en http://127.0.0.1:8310")
print(f"Documentacion: http://127.0.0.1:8310/docs")


---

## 4. APIROUTER Y MODULARIZACIÓN

### Organización por Dominio

A medida que tu API crece, mantener todos los endpoints en `main.py` se vuelve insostenible. `APIRouter` permite dividir la aplicación en módulos por dominio.

**Analogía:** Es como organizar una tienda por departamentos:
- **Sin modularización:** Todo en un pasillo gigante (confuso)
- **Con APIRouter:** Sección de ropa, electrónica, alimentos... cada una con su propio espacio

**Estructura típica:**
```
app/
├── main.py           # FastAPI principal
├── routers/
│   ├── usuarios.py   # Router de usuarios
│   ├── productos.py  # Router de productos
│   └── auth.py       # Router de autenticación
└── services/         # Servicios de lógica de negocio
```

### Ejemplo Básico de APIRouter

Creamos dos routers separados y los registramos en la app principal:

In [ ]:
# ROUTER 1: Usuarios
import uvicorn
import threading
import time
router_usuarios = APIRouter()

@router_usuarios.get("/")
def listar_usuarios():
    return {"usuarios": ["Ana", "Luis"]}

@router_usuarios.get("/{user_id}")
def obtener_usuario(user_id: int):
    return {"id": user_id, "nombre": f"Usuario{user_id}"}

# ROUTER 2: Productos
router_productos = APIRouter()

@router_productos.get("/")
def listar_productos():
    return {"productos": ["Laptop", "Mouse"]}

@router_productos.post("/")
def crear_producto(nombre: str):
    return {"mensaje": f"Producto '{nombre}' creado"}

# APP PRINCIPAL: Registra los routers
app = FastAPI()

app.include_router(router_usuarios, prefix="/usuarios", tags=["Usuarios"])
app.include_router(router_productos, prefix="/productos", tags=["Productos"])

# Pruebas
client = TestClient(app)
print(client.get("/usuarios").json())
print(client.get("/productos").json())
print(client.get("/usuarios/1").json())

# Servidor en puerto 8311
def run_server_8311():
    uvicorn.run(app, host="127.0.0.1", port=8311, log_level="warning")

server_thread = threading.Thread(target=run_server_8311, daemon=True)
server_thread.start()
time.sleep(2)

print(f"Servidor iniciado en http://127.0.0.1:8311")
print(f"Documentacion: http://127.0.0.1:8311/docs")


**Resultado:**
- `GET /usuarios/` → `listar_usuarios()`
- `GET /usuarios/{user_id}` → `obtener_usuario(user_id)`
- `GET /productos/` → `listar_productos()`
- `POST /productos/` → `crear_producto()`

### Prefijos y Tags

Los parámetros `prefix` y `tags` organizan automáticamente la documentación:

In [ ]:
import uvicorn
import threading
import time
router_auth = APIRouter(
    prefix="/auth",           # Todos los endpoints empiezan con /auth
    tags=["Autenticación"]     # Agrupación en /docs
)

@router_auth.post("/login")
def login(username: str, password: str):
    return {"token": "abc123"}

@router_auth.post("/logout")
def logout():
    return {"mensaje": "Sesión cerrada"}

app = FastAPI()
app.include_router(router_auth)

# Ahora los endpoints son:
# POST /auth/login
# POST /auth/logout

client = TestClient(app)
print(client.post("/auth/login?username=admin&password=1234").json())

# Servidor en puerto 8312
def run_server_8312():
    uvicorn.run(app, host="127.0.0.1", port=8312, log_level="warning")

server_thread = threading.Thread(target=run_server_8312, daemon=True)
server_thread.start()
time.sleep(2)

print(f"Servidor iniciado en http://127.0.0.1:8312")
print(f"Documentacion: http://127.0.0.1:8312/docs")


**En `/docs`:**
- Los endpoints aparecen agrupados bajo "Autenticación"
- Cada grupo es colapsable
- Facilita la navegación en APIs grandes

### Dependencias Globales en Router

Puedes aplicar dependencias a **todos** los endpoints de un router:

In [ ]:
# Dependencia de autenticación
import uvicorn
import threading
import time
def verificar_admin(token: str = Header(...)):
    if token != "admin-token":
        raise HTTPException(status_code=403, detail="No eres admin")
    return True

# Router con dependencia global (no necesitamos Annotated aquí porque
# la dependencia se aplica al router completo, no a parámetros individuales)
router_admin = APIRouter(
    prefix="/admin",
    tags=["Administración"],
    dependencies=[Depends(verificar_admin)]  # Aplicada a TODOS los endpoints
)

@router_admin.get("/usuarios")
def admin_listar_usuarios():
    # Ya pasó la verificación de admin
    return {"usuarios": ["todos los usuarios"]}

@router_admin.delete("/usuarios/{user_id}")
def admin_eliminar_usuario(user_id: int):
    # También requiere autenticación de admin
    return {"mensaje": f"Usuario {user_id} eliminado"}

app = FastAPI()
app.include_router(router_admin)

# Prueba
client = TestClient(app)
response = client.get("/admin/usuarios", headers={"token": "admin-token"})
print(response.json())

# Sin token falla
response_fail = client.get("/admin/usuarios")
print(f"Sin token: {response_fail.status_code}")  # 422 Unprocessable Entity

# Servidor en puerto 8313
def run_server_8313():
    uvicorn.run(app, host="127.0.0.1", port=8313, log_level="warning")

server_thread = threading.Thread(target=run_server_8313, daemon=True)
server_thread.start()
time.sleep(2)

print(f"Servidor iniciado en http://127.0.0.1:8313")
print(f"Documentacion: http://127.0.0.1:8313/docs")


**Ventaja:** No necesitas repetir `Depends(verificar_admin)` en cada endpoint del router.

### 🎯 MICRO-RETO 4: Router de Reportes con Dependencia Global

Crea un `APIRouter` para reportes que:
- Tenga prefijo `/reportes`
- Tag "Reportes"
- Dependencia global que valide header `x-departamento` == "finanzas"
- Dos endpoints: `GET /ventas` y `GET /gastos`

In [ ]:
# TODO: Implementa la dependencia de validación
import uvicorn
import threading
import time
def validar_departamento():
    pass  # Tu código aquí

# TODO: Crea el router con dependencia global
router_reportes = APIRouter(
    # Tu código aquí
)

# TODO: Implementa los endpoints
@router_reportes.get("/ventas")
def reporte_ventas():
    pass

@router_reportes.get("/gastos")
def reporte_gastos():
    pass

app = FastAPI()
# TODO: Registra el router

# Prueba (descomenta cuando termines)
# client = TestClient(app)
# print(client.get("/reportes/ventas", headers={"x-departamento": "finanzas"}).json())
# print(client.get("/reportes/ventas", headers={"x-departamento": "marketing"}).status_code)

# Servidor en puerto 8314
def run_server_8314():
    uvicorn.run(app, host="127.0.0.1", port=8314, log_level="warning")

server_thread = threading.Thread(target=run_server_8314, daemon=True)
server_thread.start()
time.sleep(2)

print(f"Servidor iniciado en http://127.0.0.1:8314")
print(f"Documentacion: http://127.0.0.1:8314/docs")


---

## 5. SUB-DEPENDENCIAS

### Composición de Dependencias

Las dependencias pueden depender de otras dependencias, creando un **grafo de resolución**. FastAPI resuelve automáticamente este grafo en el orden correcto.

**Analogía:** Es como hacer una receta de cocina:
- Para hacer **pastel** necesitas **masa** y **glaseado**
- Para hacer **masa** necesitas **harina** y **huevos**
- FastAPI "cocina" todo en el orden correcto automáticamente

### Ejemplo: Dependencias Anidadas

Creamos una cadena de dependencias donde cada una depende de la anterior:

In [ ]:
# Dependencia nivel 1: Base de datos
import uvicorn
import threading
import time
def obtener_db():
    print("1️⃣ Conectando a BD...")
    return {"conexion": "activa"}

# Tipo para BD
DatabaseDep = Annotated[dict, Depends(obtener_db)]

# Dependencia nivel 2: Repository (depende de BD)
def obtener_repository(db: DatabaseDep):
    print("2️⃣ Inicializando repository...")
    return {"db": db, "repositorio": "UsuarioRepo"}

# Tipo para Repository
RepositoryDep = Annotated[dict, Depends(obtener_repository)]

# Dependencia nivel 3: Service (depende de Repository)
def obtener_service(repo: RepositoryDep):
    print("3️⃣ Creando servicio...")
    return {"repo": repo, "servicio": "UsuarioService"}

# Tipo para Service
ServiceDep = Annotated[dict, Depends(obtener_service)]

app = FastAPI()

@app.get("/datos")
def obtener_datos(service: ServiceDep):
    print("4️⃣ Ejecutando endpoint...")
    return {"resultado": "OK", "servicio": service}

# Prueba
client = TestClient(app)
response = client.get("/datos")
print(response.json())

# Servidor en puerto 8315
def run_server_8315():
    uvicorn.run(app, host="127.0.0.1", port=8315, log_level="warning")

server_thread = threading.Thread(target=run_server_8315, daemon=True)
server_thread.start()
time.sleep(2)

print(f"Servidor iniciado en http://127.0.0.1:8315")
print(f"Documentacion: http://127.0.0.1:8315/docs")


**Flujo de ejecución:**
```
Request → /datos
FastAPI detecta: obtener_datos necesita obtener_service
FastAPI detecta: obtener_service necesita obtener_repository
FastAPI detecta: obtener_repository necesita obtener_db

FastAPI ejecuta en orden:
1. obtener_db() → {"conexion": "activa"}
2. obtener_repository(db) → {...}
3. obtener_service(repo) → {...}
4. obtener_datos(service) → respuesta
```

### Grafo de Resolución Automático

FastAPI crea un grafo dirigido acíclico (DAG) de dependencias y garantiza:
1. **Orden correcto:** Las dependencias se resuelven de abajo hacia arriba
2. **Sin duplicados:** Si dos endpoints usan la misma dependencia, FastAPI la ejecuta **una sola vez por request**
3. **Cache automático:** Los resultados se cachean durante el request

In [ ]:
# Contador para ver cuántas veces se ejecuta
import uvicorn
import threading
import time
contador = {"llamadas": 0}

def dependencia_costosa():
    contador["llamadas"] += 1
    print(f"🔄 Ejecutando dependencia costosa (llamada #{contador['llamadas']})")
    time.sleep(0.1)  # Simula operación costosa
    return {"data": "resultado"}

# Tipo reutilizable
DataCostosa = Annotated[dict, Depends(dependencia_costosa)]

def servicio_a(data: DataCostosa):
    return {"servicio": "A", "data": data}

def servicio_b(data: DataCostosa):
    # ¡Usa la MISMA dependencia!
    return {"servicio": "B", "data": data}

# Tipos para servicios
ServicioADep = Annotated[dict, Depends(servicio_a)]
ServicioBDep = Annotated[dict, Depends(servicio_b)]

app = FastAPI()

@app.get("/combinar")
def combinar_servicios(a: ServicioADep, b: ServicioBDep):
    return {"a": a, "b": b, "total_llamadas": contador["llamadas"]}

# Prueba
contador["llamadas"] = 0  # Reset
client = TestClient(app)
response = client.get("/combinar")
print(response.json())
# Observa: solo 1 llamada a dependencia_costosa (no 2)

# Servidor en puerto 8316
def run_server_8316():
    uvicorn.run(app, host="127.0.0.1", port=8316, log_level="warning")

server_thread = threading.Thread(target=run_server_8316, daemon=True)
server_thread.start()
time.sleep(2)

print(f"Servidor iniciado en http://127.0.0.1:8316")
print(f"Documentacion: http://127.0.0.1:8316/docs")


**Optimización:** FastAPI detecta que `servicio_a` y `servicio_b` usan la misma dependencia y **reutiliza el resultado**.

### 🎯 MICRO-RETO 5: Cadena de Autenticación

Crea una cadena de 3 dependencias:
1. `validar_token(token: str = Header(...))`: valida token == "secret"
2. `obtener_usuario(token_validado: str = Depends(validar_token))`: devuelve dict con usuario
3. `verificar_admin(usuario: dict = Depends(obtener_usuario))`: verifica si usuario["rol"] == "admin"

Crea endpoint `/admin/panel` que use la última dependencia.

In [ ]:
# TODO: Implementa las 3 dependencias
import uvicorn
import threading
import time
def validar_token():
    pass  # Tu código aquí

def obtener_usuario():
    pass  # Tu código aquí (debe usar Depends(validar_token))

def verificar_admin():
    pass  # Tu código aquí (debe usar Depends(obtener_usuario))

app = FastAPI()

# TODO: Implementa el endpoint
@app.get("/admin/panel")
def panel_admin():
    pass  # Usa Depends(verificar_admin)

# Prueba (descomenta cuando termines)
# client = TestClient(app)
# print(client.get("/admin/panel", headers={"token": "secret"}).json())

# Servidor en puerto 8317
def run_server_8317():
    uvicorn.run(app, host="127.0.0.1", port=8317, log_level="warning")

server_thread = threading.Thread(target=run_server_8317, daemon=True)
server_thread.start()
time.sleep(2)

print(f"Servidor iniciado en http://127.0.0.1:8317")
print(f"Documentacion: http://127.0.0.1:8317/docs")
